In [559]:
# Initialize Otter
import otter
grader = otter.Notebook("ps8.ipynb")

In [560]:
import sqlite3
import re

## Question 1: regex puzzles
Using 30 characters or less, write a regex that matches the words in the left-hand column, and does not match the words in the right-hand column.

**1(a)** (3 pts) For this question only, do not use `$` in your regex.

| **Matches** | **Doesn't match** |
|-------------|-------------------|
| owe         | silkweed          |
| overawe     | weave             |
| ewe         | well              |
| awe         | awesome           |
| we          | howitzer          |

In [561]:
regex1a='(?=^(?!.*[sl]).*we)^(?!w)'
...

In [562]:
grader.check("q1a")

q1a results: All test cases passed!

**1(b)** (3 pts)

| **Matches** | **Doesn't match** |
|-------------|-------------------|
| civic       | housing           |
| redder      | twill             |
| level       | beer              |
| foof        | iota              |
| hallah      | zoonota           |

In [563]:
regex1b = '(?=^(?!.*[bztg]))'

In [564]:
grader.check("q1b")

q1b results: All test cases passed!

**1(c)** (3 pts)

| **Matches**       | **Doesn't match** |
|-------------------|-------------------|
| dunderheadedness  | overpunishment    |
| divekeeper        | amplification     |
| homologous        | beer              |
| unintelligibility | tenderheartedness |
| locomotory        | Ternstroemiaceae  |
| elevener          | Ludgate           |

In [565]:
regex1c = '(?=^(?!.*[LTf]))^(?![bo])'

In [566]:
grader.check("q1c")

q1c results: All test cases passed!

**1(d)** (3 pts)

| **Matches** | **Doesn't match** |
|-------------|-------------------|
| effusive    | hippogriffin      |
| noisefully  | overattached      |
| disarmature | trillion          |
| suspenseful | unfitting         |
| unversatile | commotive         |
| calomorphic | abba              |

In [567]:
regex1d = '(?=^(?!.*[g]))^(?!co|[ota])'

In [568]:
grader.check("q1d")

q1d results: All test cases passed!

## Question 2: Basic SQL
In this problem, you'll interact with a toy SQL database using Python's
built-in `sqlite3` package. Documentation can be found at
<https://docs.python.org/3/library/sqlite3.html>. For this problem,
we'll use a popular toy SQLite database, called Chinook, which
represents a digital music collection. See the documentation at <https://github.com/lerocha/chinook-database/blob/master/README.md>
for a more detailed explanation. We'll use the `chinook.sqlite` file:

In [569]:
con2 = sqlite3.connect('chinook.sqlite')
con2.row_factory = lambda curs, r: r[0] # stop returning tuples!
cur = con.cursor()

**2(a)** (1 pt) Load the database using the Python `sqlite3` package. How many tables are in the database? Save the answer in the variable `n_tables`.

In [570]:
n_tables = con2.execute('SELECT count(*) FROM sqlite_master WHERE type = "table";').fetchall()[0]

In [571]:
grader.check("q2a")

q2a results: All test cases passed!

**2(b)** (2 pts) What are the names of the tables in the database? Save the answer as
    a set of strings, `table_names`. **Note:** you should write Python `sqlite3`
    code to answer this; don't just look up the answer in the
    documentation!


In [572]:
table_names = set(con2.execute('SELECT name FROM sqlite_master WHERE type = "table";').fetchall())

In [573]:
grader.check("q2b")

q2b results: All test cases passed!

**2(c)** (2 pts) Write a function `albums_starting_with(c)` that takes as an argument a single character `c` and
    returns a list of the primary keys of all the albums whose titles
    start with that character. Your function should ignore case, so that
    the inputs "a" and "A" yield the same results. Include error
    checking that raises an error in the event that the input is not a
    single character.


In [574]:
def albums_starting_with(c):
    
    assert type(c) == str and len(c) == 1
    st = c.lower() + '%'
    output = con2.execute('SELECT AlbumId FROM Album WHERE Title LIKE "%s";' %st).fetchall()

    return output
    ...

In [575]:
grader.check("q2c")

q2c results: All test cases passed!

**2(d)** (2 pts) Write a function `songs_starting_with(c)` that takes as an argument a single character and
    returns a list of the primary keys of all the **songs** whose album
    names begin with that letter. Again, your function should ignore
    case and perform error checking as in the previous exercise (again ignoring case).
    **Hint:** you'll need a JOIN statement here. Don't forget that you
    can use the `cursor.description` attribute to find out about tables
    and the names of their columns.


In [576]:
def songs_starting_with(c):
    
    assert type(c) == str and len(c) == 1
    st = c.lower() + '%'
    output = con2.execute('SELECT Track.TrackId FROM Track INNER JOIN Album ON Track.AlbumId = Album.AlbumId WHERE Title LIKE "%s";' %st).fetchall()
    
    return output
    ...

In [577]:
grader.check("q2d")

q2d results: All test cases passed!

**2(e)** (2 pts) Write a function `cost_of(c)` that takes as an argument a single character and
    returns the cost of buying every song whose album begins with that
    letter. This cost should be based on the tracks' unit prices, so
    that the cost of buying a set of tracks is simply the sum of the
    unit prices of all the tracks in the set. Again your function should
    ignore case and perform appropriate error checking.


In [578]:
def cost_of(c):
    # BEGIN SOLUTION
    assert type(c) == str and len(c) == 1
    res = con2.execute("""SELECT SUM(c.UnitPrice)
        FROM Album a
        JOIN Track b
        JOIN (SELECT DISTINCT TrackId, UnitPrice from InvoiceLine) c 
        ON a.AlbumId = b.AlbumId 
        AND b.TrackId = c.TrackId
        WHERE substr(lower(Title), 1, 1) = ?""", c.lower())
    return res.fetchall()[0]
    # END SOLUTION

In [579]:
grader.check("q2e")

q2e results: All test cases passed!

## Question 3: Additional SQL
In this case, we will use the included file `flights.sqlite`, which is a SQLite version of the `flights` dataset that we saw in Lectures 10–12.

In [580]:
con = sqlite3.connect('flights.sqlite')
cur = con.cursor()

Each question asks you to write a SQL query that performs a certain operation. For example, if the question asks, "how many flights are there", an answer could be 

> `SELECT COUNT(*) FROM flights`

Some questions refer to a parameter $x$. For these, you should write a query containing a placeholder, which we can run for different values of $x$. For example, if the question asks, "how many flights are there in month $x$?", where $x\in\{1,2,\dots,12\}$ is a numerical month, an answer could be 

> `SELECT COUNT(*) FROM flights WHERE month=?`.

For answers that ask for a single datum, we will run 
```
> cur.execute(your_query)
> result = cur.fetchone()
```
For answers that ask for a table or set of rows, we will use `cur.fetchall(...)` instead of `cur.fetchone(...)`. For answers that accept a parameter, we will use e.g. `cur.fetchone(..., [x])`.

**3(a)** (2 pts) What was the number of scheduled departures on day of the week $x$, where $x$ is the numerical day of the week (Sunday=0, Monday=1, etc.) Your query should return a tuple `(n, x)` where `n` is the number of flights, and `x` is the day of week.

In [581]:
avg_sched_dep = """SELECT COUNT(*), cast(strftime("%w", date(datetime(time_hour, "unixepoch"))) as integer) as dw 
                     from flights
                     where dw=cast(? as text);
                """

In [582]:
grader.check("q3a")

q3a results: All test cases passed!

**3(b)** (2 pts) Which month had the most scheduled departures? Your query should return tuple containing a single numerical month (for example `(2,)` if the answer is February.)

In [583]:
most_deps = """
            SELECT month 
            FROM flights  GROUP BY month
            HAVING COUNT(month)=(SELECT MAX(cnt) 
            FROM( SELECT month, COUNT(month) cnt
            FROM flights
            GROUP BY month));
            """
#print(con.execute(most_deps).fetchall())

In [584]:
grader.check("q3b")

q3b results: All test cases passed!

**3(c)** (3 pts) On how many days was the highest recorded temperature below $x$ degrees?

In [585]:
high_above = """SELECT COUNT(*) FROM (SELECT MAX(temp) temp FROM weather GROUP BY year, month, day) WHERE temp<?;"""


#table_names = set(con.execute('SELECT name FROM sqlite_master WHERE type = "table";').fetchall())
#print(table_names)
#print(con.execute('PRAGMA table_info(weather);').fetchall())
#print(con.execute("""SELECT * FROM (SELECT MAX(temp) temp FROM weather GROUP BY year, month, day)""").fetchall())
#print(con.execute("""SELECT COUNT(*) FROM flights""").fetchall())

In [586]:
grader.check("q3c")

q3c results: All test cases passed!

**3(d)** (3 pts) In this dataset, a cancelled flight is one for which `dep_delay` is missing. What percentage of the flights were cancelled overall?

In [587]:
pct_cancelled = 'SELECT (COUNT(*) - COUNT(dep_delay)) / cast(COUNT(*) as real) FROM flights;'

In [588]:
grader.check("q3d")

q3d results: All test cases passed!

**3(e)** (4 pts) What percentage of flights were cancelled on days where the max temperature was a) at least 35 degrees, versus b) less than 35 degrees? Your query should return a table that looks like

| warm | pct_cancelled |
|------|---------------|
|0     | $.xxx$        |
|1     | $.yyy$        |

In [1]:
pct_cancelled_by_temp = """
                        SELECT (CASE WHEN max_t > 35 THEN 1 ELSE 0 END) warm, ((COUNT(*) - COUNT(dep_delay)) / cast(COUNT(*) as real)) pct_cancelled
                        FROM (
                        SELECT flights.year, flights.month, flights.day, max_t, dep_delay FROM flights
                        INNER JOIN (
                        SELECT year, day, month, max_t FROM (SELECT year, month, day, MAX(temp) max_t FROM weather GROUP BY year, month, day)) weather2
                        ON weather2.day = flights.day AND weather2.year = flights.year AND weather2.month = flights.month)
                        GROUP BY warm;
                        """

In [2]:
grader.check("q3e")

NameError: name 'grader' is not defined

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [591]:
grader.check_all()

q1a results: All test cases passed!

q1b results: All test cases passed!

q1c results: All test cases passed!

q1d results: All test cases passed!

q2a results: All test cases passed!

q2b results: All test cases passed!

q2c results: All test cases passed!

q2d results: All test cases passed!

q2e results: All test cases passed!

q3a results: All test cases passed!

q3b results: All test cases passed!

q3c results: All test cases passed!

q3d results: All test cases passed!

q3e results: All test cases passed!

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Upload this .zip file to Gradescope for grading.

In [592]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False)